In [1]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset_tv = pd.read_csv('2018-07-12_161320_mentions_by_media_type.csv')
dataset_radio = pd.read_csv('2018-07-12_161430_mentions_by_media_type.csv')
dataset_online_news = pd.read_csv('ONLINE-NEWS.csv')
dataset_stocks = pd.read_csv('MSFT.csv')

#removing excess information and prepairing the training dataset
dataset_radio = dataset_radio.iloc[0:31,2:3]
dataset_onlinenews = dataset_online_news.iloc[0:31,2:3]
dataset_tv = dataset_tv.iloc[0:31,2:3]
dataset_stocks = dataset_stocks.iloc[0:31,1:2]


In [3]:
dataset_total = pd.concat((dataset_tv['MENTIONS'], dataset_radio['MENTIONS'], dataset_onlinenews['MENTIONS'], dataset_stocks['Open']), axis = 1)


In [4]:
#feature scaling 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
dataset_scaled = sc.fit_transform(dataset_total)


In [5]:

Training_dataset_scaled = dataset_scaled[0:25,0:4]

test_dataset_scaled = dataset_scaled[25:31,0:4]


In [6]:
# Creating a data structure with 5 timesteps and 1 output
x_train = []
y_train = []
for i in range(5, 25):
    x_train.append(Training_dataset_scaled[i-5:i,0:4])
    y_train.append(Training_dataset_scaled[i,3])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train.shape[0]
#x_train.shape[1]
#x_train.shape[2]

20

In [7]:
## reshaping
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],x_train.shape[2]))


In [8]:
#Building the RNN

#importing the keras library and pakages
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM

#initialising the rnn
regressor = Sequential() 

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20, return_sequences = True, input_shape = (x_train.shape[1],x_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 20))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

C:\Users\Shalini\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# Fitting the RNN to the Training set
regressor.fit(x_train, y_train, epochs = 125, batch_size = 32)


Epoch 1/125
20/20 [==============================] - 5s 243ms/step - loss: 0.1437
Epoch 2/125
20/20 [==============================] - 0s 2ms/step - loss: 0.1355
Epoch 3/125
20/20 [==============================] - 0s 2ms/step - loss: 0.1276
Epoch 4/125
20/20 [==============================] - 0s 1ms/step - loss: 0.1175
Epoch 5/125
20/20 [==============================] - 0s 4ms/step - loss: 0.1156
Epoch 6/125
20/20 [==============================] - 0s 1ms/step - loss: 0.1074
Epoch 7/125
20/20 [==============================] - 0s 832us/step - loss: 0.0975
Epoch 8/125
20/20 [==============================] - 0s 909us/step - loss: 0.0882
Epoch 9/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0793
Epoch 10/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0805
Epoch 11/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0727
Epoch 12/125
20/20 [==============================] - 0s 860us/step - loss: 0.0702
Epoch 13/125
20/20 [=========

Epoch 101/125
20/20 [==============================] - 0s 780us/step - loss: 0.0214
Epoch 102/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0133
Epoch 103/125
20/20 [==============================] - 0s 646us/step - loss: 0.0170
Epoch 104/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0186
Epoch 105/125
20/20 [==============================] - 0s 884us/step - loss: 0.0159
Epoch 106/125
20/20 [==============================] - 0s 836us/step - loss: 0.0140
Epoch 107/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0178
Epoch 108/125
20/20 [==============================] - 0s 987us/step - loss: 0.0182
Epoch 109/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0179
Epoch 110/125
20/20 [==============================] - 0s 1ms/step - loss: 0.0171
Epoch 111/125
20/20 [==============================] - 0s 838us/step - loss: 0.0155
Epoch 112/125
20/20 [==============================] - 0s 801us/step - loss: 0.0170
Ep

In [30]:
inputs = dataset_scaled[20:31,0:4]

In [31]:
x_test = []
for j in range(26, 31):
    x_test.append(dataset_scaled[j-5:j, 0:4])

In [32]:
#x_test[1]
#x_test = x_test.reshape(1,-1)

In [33]:
x_test = np.array(x_test)

In [44]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))


In [45]:
x_test.shape[1]

5

In [46]:
predicted_stock_price = regressor.predict(x_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
print(predicted_stock_price)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.